In [2]:
import torch
import torch.nn as nn
import numpy as np


class LongShortTermMemoryModel(nn.Module):
    def __init__(self, encoding_size):
        super(LongShortTermMemoryModel, self).__init__()

        self.lstm = nn.LSTM(encoding_size, 128)  # 128 is the state size
        self.dense = nn.Linear(128, encoding_size)  # 128 is the state size

    def reset(self):  # Reset states prior to new input sequence
        zero_state = torch.zeros(1, 1, 128)  # Shape: (number of layers, batch size, state size)
        self.hidden_state = zero_state
        self.cell_state = zero_state

    def logits(self, x):  # x shape: (sequence length, batch size, encoding size)
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        return self.dense(out.reshape(-1, 128))

    def f(self, x):  # x shape: (sequence length, batch size, encoding size)
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):  # x shape: (sequence length, batch size, encoding size), y shape: (sequence length, encoding size)
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

emojis_mapping = {
  'hat': '🎩',
  'rat': '🐀',
  'cat': '🐈',
  'flat': '🏠',
  'matt': '👱',
  'cap': '🧢',
  'son': '👶',
}


index_to_emoji =[value for _,value in emojis_mapping.items()]

index_to_char = [' ', 'h', 'a', 't', 'r','c', 'f', 'l', 'm', 'p', 's', 'o', 'n']

char_encodings = np.eye(len(index_to_char))
encoding_size = len(char_encodings)
emojies = np.eye(len(emojis_mapping))
emoji_encoding_size=len(emojies)

letters = {}

for i, letter in enumerate(index_to_char):
        letters[letter] = char_encodings[i]

x_train = torch.tensor([
        [[letters['h']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['r']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['c']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['f']], [letters['l']], [letters['a']], [letters['t']]],
        [[letters['m']], [letters['a']], [letters['t']], [letters['t']]],
        [[letters['c']], [letters['a']], [letters['p']], [letters[' ']]],
        [[letters['s']], [letters['o']], [letters['n']], [letters[' ']]],
        ], dtype=torch.float)


y_train = torch.tensor([
        [emojies[0], emojies[0], emojies[0], emojies[0]] ,
        [emojies[1], emojies[1], emojies[1], emojies[1]],
        [emojies[2], emojies[2], emojies[2], emojies[2]],
        [emojies[3], emojies[3], emojies[3], emojies[3]],
        [emojies[4], emojies[4], emojies[4], emojies[4]],
        [emojies[5], emojies[5], emojies[5], emojies[5]],
        [emojies[6], emojies[6], emojies[6], emojies[6]]], dtype=torch.float)

model = LongShortTermMemoryModel(encoding_size)

optimizer = torch.optim.RMSprop(model.parameters(), 0.001)
for epoch in range(500):
    for i in range(x_train.size()[0]):
        model.reset()
        model.loss(x_train[i], y_train[i]).backward()
        optimizer.step()
        optimizer.zero_grad()


In [12]:

def emoji(input):
    y = -1
    model.reset()
    for i in range(len(input)):
        char_index = index_to_char.index(input[i])
        y = model.f(torch.tensor([[char_encodings[char_index]]], dtype=torch.float))
    return index_to_emoji[y.argmax(1)]

print(emoji('rats'))
print(emoji('rt'))
print(emoji('fla'))
print(emoji('mt'))
print(emoji('sn'))
print(emoji('ht'))


🐀
🐀
🏠
👱
👶
🎩
